# Prueba con Kapacitor custom UDFs

Mock para inyectar valores de prueba en InfluxDB

In [1]:
import threading
import time
import math
import datetime
from influxdb import InfluxDBClient

def ValueGenerator(w=0.01,a=10.0, phi=0.0):
    t = 0.0
    sampling_freq = 2.0
    while True:
        y = a*math.sin( w * (t/sampling_freq) * (2*math.pi) + phi )
        t = t + 1.0
        yield y

class InfluxDBPublisher(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.keep_running = True
        self.client = InfluxDBClient(host='influxdb', port=8086, username='root', password='root',
                                     database="my_application")

    def run(self):
        i = 0
        gens = [ 
            ValueGenerator(w=0.01,a=10.0, phi=0.0),
            ValueGenerator(w=0.02,a=20.0, phi=0.1), 
            ValueGenerator(w=0.03,a=30.0, phi=0.2),
            ValueGenerator(w=0.04,a=40.0, phi=0.3),
            ValueGenerator(w=0.05,a=50.0, phi=0.4)
        ]
        while self.keep_running:
            time.sleep(0.5)
            json_body = []
            i = 0
            ts = datetime.datetime.utcnow()
            for g in gens:                
                json_body.append(
                    {
                        "measurement": f"TMV_VAR{i}",
                        "time": ts,
                        "fields": {
                            "value": next(g)
                        }
                    }
                )
                i += 1            
            self.client.write_points(json_body)

    def stop(self):
        self.keep_running = False
        self.join()


pub = InfluxDBPublisher()
pub.start()
input("Press any key to stop")
pub.stop()

Press any key to stop
